In [23]:
import pandas as pd

In [24]:
result=pd.read_csv("/content/kaldi_asr_out.csv")

In [25]:
results = result.rename(columns={'output': 0, 'ref': 1, 'translate': 2})

In [26]:
results[0]

0       list it does not mean that the functions that ...
1       you would have for instance a function like sa...
2       a l pass to now this call this this function i...
3       delete max defined for keeps but its call like...
4       it max normal the how the priority queue is ac...
                              ...                        
1731    back of declarations makes things a little bit...
1732    many come across in beginning program the othe...
1733    this is more to do convenience and representat...
1734    push say we have queue maybe also a list but w...
1735    in x and y and started manipulating p dot x an...
Name: 0, Length: 1736, dtype: object

In [27]:
!pip install torchmetrics
import torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
wer = torchmetrics.functional.word_error_rate(preds=results[0], target=results[1])
wer

tensor(0.3563)

In [29]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch = re.sub(chars_to_ignore_regex, '', batch).lower() + " "
    return batch
results[1] = results[1].map(remove_special_characters)

In [30]:
mypreds=[]
mytarget=[]
for i in range(len(results)):
  if(type(results[0][i])==str):
    mypreds.append(results[0][i])
    mytarget.append(results[1][i])

In [31]:
eval = torchmetrics.functional.word_error_rate(preds=mypreds, target=mytarget)
eval

tensor(0.2979)

In [32]:
import nltk
import string
from nltk.metrics import edit_distance

In [33]:
def similarity(word1, word2):
    # Compute the similarity score between two words based on their edit distance
    distance = edit_distance(word1, word2)
    return 1 / (distance + 1)

In [34]:
def align_words(asr_words, ref_words, similarity):
    # Initialize the alignment matrix
    alignment_matrix = [[0 for j in range(len(ref_words) + 1)] for i in range(len(asr_words) + 1)]

    # Fill in the alignment matrix using the Needleman-Wunsch algorithm
    for i in range(1, len(asr_words) + 1):
        for j in range(1, len(ref_words) + 1):
            match_score = alignment_matrix[i-1][j-1] + similarity(asr_words[i-1], ref_words[j-1])
            delete_score = alignment_matrix[i-1][j] - 1
            insert_score = alignment_matrix[i][j-1] - 1
            alignment_matrix[i][j] = max(match_score, delete_score, insert_score)

    # Trace back through the alignment matrix to get the alignment
    alignment = []
    i, j = len(asr_words), len(ref_words)
    while i > 0 and j > 0:
        if alignment_matrix[i][j] == alignment_matrix[i-1][j-1] + similarity(asr_words[i-1], ref_words[j-1]):
            alignment.append((i-1, j-1))
            i -= 1
            j -= 1
        elif alignment_matrix[i][j] == alignment_matrix[i-1][j] - 1:
            i -= 1
        else:
            j -= 1

    # Reverse the alignment and return it
    alignment.reverse()
    return alignment

In [35]:
aligned_asr=[]
for i in range(len(mypreds)):
  
  asr_words = mypreds[i].translate(str.maketrans('', '', string.punctuation)).lower().split()
  ref_words = mytarget[i].translate(str.maketrans('', '', string.punctuation)).lower().split()


  # Align the words in the ASR output and reference text
  alignment = align_words(asr_words, ref_words, similarity)

  # Apply the edit distance matching rule to correct disfluencies
  for m, n in alignment:
      if len(asr_words[m]) > 4 and edit_distance(asr_words[m], ref_words[n]) == 1:
          asr_words[m] = ref_words[n]

  # Join the corrected words back into a single string
  corrected_text = " ".join(asr_words)


  corrected_asr_output = ' '.join(asr_words)
  aligned_asr.append(corrected_asr_output)

In [36]:
eval2 = torchmetrics.functional.word_error_rate(preds=aligned_asr, target=mytarget)
eval2

tensor(0.2906)

In [37]:
for i in range(len(aligned_asr)):
  
  asr_words = aligned_asr[i].translate(str.maketrans('', '', string.punctuation)).lower().split()

  prev_word = ''
  remove_indices = []
  for j in range(len(asr_words)):
    if asr_words[j] == prev_word:
      remove_indices.append(j)
    prev_word = asr_words[j]

  for index in sorted(remove_indices, reverse=True):
    asr_words.pop(index)

  aligned_asr[i] = ' '.join(asr_words)

In [38]:
eval3 = torchmetrics.functional.word_error_rate(preds=aligned_asr, target=mytarget)
eval3

tensor(0.2900)

In [39]:
import csv

with open('processed_asr_kaldi.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(aligned_asr, mytarget))